# Solid state matter as a set of quantum harmonic oscillators

Adrien Claret-Tournier

Here we explore how matter reacts at an atomic level to changes in temperature, and how each atom can be assimilated to a harmonic oscillator vibrating with potential energy proportional to the square root of the input temperature.

In [1]:
%%capture
#### Import all necessary libraries, objects etc. ####
# plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

# animation
from matplotlib import animation
from IPython.display import HTML
!conda install -y -q ffmpeg

# interaction
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

# maths
import numpy as np

# Suppress runtime warnings
import warnings
warnings.filterwarnings('ignore')

We will first consider a system of $N_{tot}$ quantum oscillators, defined such that $N_{tot} \gg 1$ and $Q_{tot} \gg 1$. For each single oscillator with energy $E_q$, we have an average energy $\langle E \rangle$, which can be rewritten in terms of the partition function $Z$:

$$
\langle E \rangle = \frac{1}{Z} \sum_{q=0}^{\infty} E_q e^{-\beta E_q} =-\frac{1}{Z}\frac{\partial Z}{\partial \beta} = -\frac{\partial lnZ}{\partial \beta}
$$

For a Quantum Harmonic Oscillator (QHO), the energy of a quantum $q$ is $E_q = (\frac{1}{2} + q) \hbar \omega$ ; we can therefore find the partition function by approximating the upper limit of the sum with inifity, given that in our system $Q_{tot} \gg 1$.

$$
Z = e^{-\frac{1}{2}\beta \hbar \omega} \sum_{q=0}^{\infty} e^{-\beta q \hbar \omega} = \frac{1}{2\sinh(\frac{1}{2} \beta \hbar \omega)}
$$

We can now find an expression for the average energy:

$$
\langle E \rangle = \frac{\hbar \omega}{2\tanh(\frac{1}{2} \beta \hbar \omega)}
$$

In [2]:
%%capture
# Init data for animation
Nf = 30 # no. frames  ## change to ~20
dt = 50 # frame time (ms)
ballsize = 0.03

# init data for plots
kb = 1.38e-23
hbar = 1.055e-34
T = np.arange(0.1,100, 1)
w = 5e12 # ballpark frequency
b = 1/(kb*T)

# init average energy QHO
def E(x):
    return 0.5*hbar*w / np.tanh(0.5*hbar*w/(kb*x))

# init polynomial
x = np.linspace(-1, 1,Nf)
def y(a):
    return a**2

# calculate average of polynomial
def avg(sc):
    avg = 0
    for i in x:
        avg += y(sc*x)
    avg = avg/len(x)
    return avg

# init temperatures and scaling factors for example energies
T1 = 0.95                     # chosen arbitrarily
T2 = 1.75                     # ""
T3 = 2.1                      # ""

sc1 = 0.4                     # ""
sc2 = sc1 * np.sqrt(T2 / T1)  # Calculated to represent correct conversion between T and E
sc3 = sc1 * np.sqrt(T3 / T1)  # ""


# calculate position of ball depending on index i of animation (from 0 to Nf-1)     
def move(sc, i):
    # sorting out quadrants to map i between 0 and 199 to -1 and 1 (and back)
    if (i < (Nf/4-1)):
        px = sc * np.sin(np.pi/2 * (i / (Nf/4 - 1) - 1))
    if (Nf/4 < i < (Nf/2 - 1)):
        px = sc * np.sin(np.pi/2 * (i / (Nf/4) - 1) + np.pi/2)
    if (Nf/2 < i < (3*Nf/2 - 1)):
        px = sc * np.sin(np.pi/2 * ((i - Nf/2) / (Nf/4) + 1) + np.pi)
    else:
        px = sc * np.sin(np.pi/2 * (-(i - Nf/2) / (Nf/4) + 1) + np.pi)

    py = y(px) + ballsize # y-coordinates
    
    pos = [-px,py] # direction correction
    Ux.clear()
    return pos


# Set up figure and plot
QHO = plt.figure(figsize=(12,6), constrained_layout=True)
grid = QHO.add_gridspec(1, 2) # create a grid to arrange subplots
Ux = QHO.add_subplot(grid[0,0]) # harmonic oscillator
avgE = QHO.add_subplot(grid[0,1]) # average energy plot

redLabel = "$T_1=$" + str.format('{0:0.1f}', T1 * hbar*w/kb) + "K"
greenLabel = "$T_2=$" + str.format('{0:0.1f}', T2 * hbar*w/kb) + "K"
blueLabel = "$T_3=$" + str.format('{0:0.1f}', T3 * hbar*w/kb) + "K"

# Initialization function: plot the background of each frame
def init():
    Ux.clear()
    
    # Static graph: doesn't need to be called in animate()
    avgE.clear()
    avgE.set_xlabel('$T k_B / \hbar \omega$')
    avgE.set_ylabel('$E / \hbar \omega$')
    avgE.set_title('Average energy (QHO)')
    avgE.plot(T*kb/(hbar*w),E(T)/(hbar*w))
    
    avgE.plot(T1, E(T1 * hbar*w/kb)/(hbar*w), marker='o',color= 'r', label = redLabel)
    avgE.plot(T2, E(T2 * hbar*w/kb)/(hbar*w), marker='o',color= 'g', label = greenLabel)
    avgE.plot(T3, E(T3 * hbar*w/kb)/(hbar*w), marker='o',color= 'b', label = blueLabel)
    avgE.legend()


# Animation function which updates figure data
def animate(i):
    # calculate ball positions
    pos1 = move(sc1, i)
    pos2 = move(sc2, i)
    pos3 = move(sc3, i)
    
    # plot results
    Ux.set_xlabel("Displacement")
    Ux.set_ylabel("Potential energy")
    Ux.set_title('Harmonic oscillator,\narbitrary units')
    Ux.plot(x, y(x))
    Ux.add_artist(plt.Circle(pos1,ballsize, color = 'r'))
    Ux.add_artist(plt.Circle(pos2,ballsize, color = 'g'))
    Ux.add_artist(plt.Circle(pos3,ballsize, color = 'b'))
    Ux.plot([-np.sqrt(sc1**3 *2/3) - 2*ballsize, np.sqrt(sc1**3 *2/3) + 2*ballsize],
            [sc1**3 *2/3 + ballsize, sc1**3 *2/3 + ballsize], 'r-')
    
    Ux.plot([-np.sqrt(sc2**3 *2/3) - ballsize, np.sqrt(sc2**3 *2/3) + ballsize],
            [sc2**3 *2/3 + ballsize,sc2**3 *2/3 + ballsize], 'g-')
    
    Ux.plot([-np.sqrt(sc3**3 *2/3) - ballsize, np.sqrt(sc3**3 *2/3) + ballsize],
            [sc3**3 *2/3 + ballsize,sc3**3 *2/3 + ballsize], 'b-')

Below is plotted the average energy per temperature (right), and we consider 3 specific points. These points will have a potential energy proportional to the square root of their temperature. The harmonic oscillators (left) represent this, where their amplitudes are each scaled to increase by factors of $ \sqrt{T} $. The horizontal lines on the QHO plot indicates the average energy of the oscillator.

In [3]:
# Call the animator
anim = animation.FuncAnimation(QHO, animate, init_func=init,frames=Nf, interval=dt)

# Call function to display the animation
HTML(anim.to_html5_video())

### Considering Heat Capacity

We can express the average energy within a solid as:

$$
\langle E \rangle = 3N \frac{\hbar \omega}{2\tanh(\frac{1}{2} \beta \hbar \omega)}
$$

And the heat capacity:
$$
C = \frac{d\langle E \rangle}{dt} = \frac{3N \hbar^2 \omega_E^2} {4k_B T^2 \sinh^2 (\frac{\hbar \omega_E}{k_B T})}
$$

Where the Einstein frequency is defined as $\omega_E = \omega$.

Here will be represented longitudinal and transverse virbations in a lattice of particles. The amplitude of the vibrations are proportional to the square root of the average energy, which is dependent on temperature.

In [4]:
# Define constants, data and functions relevant for both interaction and animation

# Init data for animation
Nf = 200 # no. frames
dt = 20 # frame time (ms)

# init data for plots
kb = 1.38e-23
hbar = 1.055e-34
T = np.arange(0.1,100, 1)
w = 5e12 # ballpark Einstein frequency
Te = hbar*w / kb
N = 1
# Heat capacity
def C(T):
    return (3*(hbar*Te)**2) / (4 * T**2 * np.sinh(hbar*w / (kb*T))**2)

# vibration amplitude
def vibAmp(T):
    return np.sqrt(T)*0.04

In [5]:
@interact (T1=(1,100,1))
def slide(T1):  
    # Set up figure and plot
    CvPlots = plt.figure(figsize=(12,6), constrained_layout=True)
    grid = CvPlots.add_gridspec(1, 2) # create a grid to arrange subplots
    Cv = CvPlots.add_subplot(grid[0,0]) # heat capacity
    CvVib = CvPlots.add_subplot(grid[0,1]) # vibration

    Cv.set_xlabel("$T$")
    Cv.set_ylabel("$C_v/Nk_B$")

    Cv.plot(T,C(T))
    Cv.plot(T1, C(T1), marker='o', color='r')
    
    amp = vibAmp(T1)
    CvVib.add_artist(plt.Circle((0.5,0.5), amp + 0.05, color = '#e9b9b9'))
    CvVib.add_artist(plt.Circle((0.5,0.5), amp + 0.05, color = '#d47777', fill=False))
    
    CvVib.add_artist(plt.Circle((0.5 + amp,0.5), 0.05, color = '#e09e9e'))
    CvVib.add_artist(plt.Circle((0.5 - amp,0.5), 0.05, color = '#e09e9e'))
    CvVib.add_artist(plt.Circle((0.5 + 3*amp/4,0.5), 0.05, color = '#d98787'))
    CvVib.add_artist(plt.Circle((0.5 - 3*amp/4,0.5), 0.05, color = '#d98787'))
    CvVib.add_artist(plt.Circle((0.5 + amp/2,0.5), 0.05, color = '#d47777'))
    CvVib.add_artist(plt.Circle((0.5 - amp/2,0.5), 0.05, color = '#d47777'))
    CvVib.add_artist(plt.Circle((0.5 + amp/4,0.5), 0.05, color = '#ce6464'))
    CvVib.add_artist(plt.Circle((0.5 - amp/4,0.5), 0.05, color = '#ce6464'))

    CvVib.add_artist(plt.Circle((0.5,0.5), 0.05, color = 'r'))

interactive(children=(IntSlider(value=50, description='T1', min=1), Output()), _dom_classes=('widget-interact'…

The gradient of coloured balls on the right represents the increase in amplitude of the oscillations of the atom, with the largest ball being the amplitude of the atom at temperature $T_1$.

Below is animated the same principle, as an animated and saveable loop.

In [6]:
%%capture
# Init data for animation
Nf = 70 # no. frames
dt = 35 # frame time (ms)

# init plots
# Set up figure and plot
CvPlots = plt.figure(figsize=(12,6), constrained_layout=True)
grid = CvPlots.add_gridspec(1, 2) # create a grid to arrange subplots
Cv = CvPlots.add_subplot(grid[0,0]) # heat capacity
CvVib = CvPlots.add_subplot(grid[0,1]) # vibration

# init animation data
CvPoints = np.linspace(5, 80, Nf)

scmax = vibAmp(CvPoints[Nf - 1])
sineNf = np.linspace(-1,1, Nf)

def init():
    Cv.clear()
    CvVib.clear()

    
def animate(i):
    Cv.clear()
    CvVib.clear()
    
    sc = vibAmp(CvPoints[i])
    vibPoints = sc * np.sin(np.linspace(0,4*2*np.pi,Nf))

    
    # Cv plot with animated movement
    Cv.plot(T/Te, C(T))
    Cv.plot(CvPoints[i]/Te, C(CvPoints[i]), marker='o', color='r')
    
    
    #CvVib.set_ylim(0,0.5)
    CvVib.add_artist(plt.Circle((0.5 + vibPoints[i], 4*y(vibPoints[i]) + 0.01), 0.01, color='r'))
    CvVib.plot(0.5 + scmax * sineNf, 4*y(scmax * sineNf))

In [7]:
# Call the animator
anim = animation.FuncAnimation(CvPlots, animate, init_func=init,frames=Nf, interval=dt)

# Call function to display the animation
HTML(anim.to_html5_video())

# Wave propagation: phonons and the heat capacity of solids

For a solid consisting of N oscillators that can each vibrate in 3D, we can  distinguish between two types of energy propagation: longitudinally and transverse. These two methods of propagation can be distinguished by how the energy they carry is transferred across an atomic lattice, or material. Looking at a 2D example, the animation below shows both types of propagation. This can be radily extended to 3D, where the lattice allows for 2 longitudinal waves and 1 transverse, for 3 degrees of freedom in the solid.

In [8]:
%%capture
#### Transverse and Longitudinal vibrations ####

# init animation parameters
Nf = 75 # no. frames
dt = 45 # frame time (ms)

# Set up figure and plot
vibes = plt.figure(figsize=(12,8), constrained_layout=True)
grid = vibes.add_gridspec(2, 2) # create a grid to arrange subplots
long = vibes.add_subplot(grid[0,0], title = "Longitudinal wave") # Longitudinal perturbation
trans = vibes.add_subplot(grid[0,1], title = "Transverse wave") # Transverse perturbation
oneDchain = vibes.add_subplot(grid[1,:], title = "longitudinal and transverse waves")


# init values for 2D wave propagation
nparticles = 13 # no. of particles per side. Works best between 10-20

animPortion = 7 # what fraction of the total runtime will the wave be visible for

sc = 0.6/nparticles # amplitude of wave

sine = sc * np.sin(np.linspace(0,2*np.pi, int((Nf)/animPortion))) # governs size of sine wave in lattice

# array to build total function that changes lattice shape (straight line - sine - straight line:  -~- )
linear = np.zeros(int(Nf * (animPortion-1)/animPortion))
endLinear = np.zeros(nparticles)
pert = np.append(linear, sine)
perturbation = np.append(pert, endLinear)

# create simple arrays
xsl = np.linspace(0, 1, nparticles)
ysl = np.ones(nparticles)
xst = np.linspace(0, 1, nparticles)
yst = np.ones(nparticles)


# init animation frame etc
def initVibes():
    long.clear()
    trans.clear()
    oneDchain.clear()

# Animation function which updates data
def animateVibes(frame):
    long.clear()
    trans.clear()
    oneDchain.clear()
       
    long.set_ylim(-0.1,1.1)
    trans.set_xlim(-0.1,1.1)
    oneDchain.set_ylim(0.9,1.1)
    oneDchain.set_xlim(-0.1,1.1)
    
    i=0
    j=0
    while j < nparticles: # builds vertical arrays
        while i < nparticles: # builds horizontal arrays
            ysl[nparticles - 1 - i] = 1 + perturbation[frame + i] #offsets ys by position on function given by frame number
            xst[nparticles - 1 - i] = xst[nparticles - 1 - i] + perturbation[frame + i]
            i += 1
        # offsets ys by rank of xs, building 2D lattice
        long.scatter(xsl, (ysl - xsl[j]), color = 'b')
        trans.scatter(xst, (yst - xsl[j]), color = 'b')
        j += 1
        oneDchain.scatter(xst, ysl)
        

In [9]:
# Call the animator
anim = animation.FuncAnimation(vibes, animateVibes, init_func=initVibes,frames=Nf, interval=dt)

# Call function to display the animation
HTML(anim.to_html5_video())